---

## 11. 최종 요약 및 다음 단계

### 📊 Part 3 핵심 성과

✅ **Data Leakage 완전 제거**
- Test Set은 최종 평가 단 한 번만 사용
- 모든 의사결정은 Train/Validation에서만 수행

✅ **3-Way Split 성공**
- Train (60%) / Validation (20%) / Test (20%)
- 모든 세트에서 부도율 균일 유지

✅ **리샘플링 전략 선택**
- SMOTE vs Class Weight 대조 실험
- Validation Set 기반 선택

✅ **최적 모델 선택**
- 4개 모델 튜닝 및 비교
- Validation Set 기반 최종 선택

✅ **임계값 최적화**
- F2-Score 최적 임계값
- Recall 80% 보장 임계값
- Validation Set에서만 결정

✅ **Traffic Light 시스템**
- 데이터 기반 임계값 (Recall 80%/95%)
- 비즈니스 의사결정 지원

✅ **일반화 성능 확인**
- Test Set PR-AUC, Recall, F2-Score
- Traffic Light 리스크 방어율

### 🚀 다음 단계 (Part 4)

- SHAP 분석을 통한 모델 해석
- 개별 예측 설명
- 비즈니스 인사이트 도출

---

**노트북 작성 완료!** 🎉

In [ ]:
# 저장할 파일들
save_files = {
    # 모델
    f'{OUTPUT_PREFIX}_최종모델.pkl': final_model,
    f'{OUTPUT_PREFIX}_분류기.pkl': classifier,  # SHAP용
    
    # 데이터
    f'{OUTPUT_PREFIX}_X_train.csv': X_train,
    f'{OUTPUT_PREFIX}_X_val.csv': X_val,
    f'{OUTPUT_PREFIX}_X_test.csv': X_test,
    f'{OUTPUT_PREFIX}_y_train.csv': pd.DataFrame({'target': y_train}),
    f'{OUTPUT_PREFIX}_y_val.csv': pd.DataFrame({'target': y_val}),
    f'{OUTPUT_PREFIX}_y_test.csv': pd.DataFrame({'target': y_test}),
    
    # 임계값 및 결과
    f'{OUTPUT_PREFIX}_임계값.pkl': {
        'selected_threshold': selected_threshold,
        'red_threshold': red_threshold,
        'yellow_threshold': yellow_threshold,
        'f2_optimal_threshold': f2_optimal_threshold
    },
    
    f'{OUTPUT_PREFIX}_결과요약.pkl': {
        'model_name': final_model_name,
        'validation_pr_auc': best_val_pr_auc,
        'test_pr_auc': test_pr_auc,
        'test_recall': test_recall,
        'test_precision': test_precision,
        'test_f2_score': test_f2_score,
        'test_type2_error': test_type2_error,
        'risk_captured_test': risk_captured_test
    }
}

# 파일 저장
print("=" * 70)
print("💾 파일 저장 중...")
print("=" * 70)

for filename, data in save_files.items():
    filepath = os.path.join(PROCESSED_DIR, filename)
    
    if filename.endswith('.pkl'):
        joblib.dump(data, filepath)
    elif filename.endswith('.csv'):
        data.to_csv(filepath, index=False, encoding='utf-8-sig')
    
    print(f"✅ {filename}")

print("=" * 70)
print(f"\\n📂 저장 위치: {PROCESSED_DIR}")
print("✅ 모든 파일 저장 완료!")
print("=" * 70)

---

## 10. 모델 및 결과 저장

### Part 4 SHAP 분석을 위한 파일 저장

In [ ]:
# Feature Importance 추출 (모델에 따라 다름)
classifier = final_model.named_steps['classifier']

if hasattr(classifier, 'feature_importances_'):
    # Tree 기반 모델 (LightGBM, XGBoost, CatBoost, RandomForest)
    importance = classifier.feature_importances_
    feature_names = X_train.columns
elif hasattr(classifier, 'coef_'):
    # 선형 모델 (Logistic Regression)
    importance = np.abs(classifier.coef_[0])
    feature_names = X_train.columns
else:
    print("⚠️  Feature Importance를 추출할 수 없는 모델입니다.")
    importance = None
    feature_names = None

if importance is not None:
    # Feature Importance DataFrame
    feat_imp_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance
    }).sort_values('Importance', ascending=False).reset_index(drop=True)
    
    print("=" * 70)
    print(f"🏆 Top 15 Feature Importance ({final_model_name})")
    print("=" * 70)
    print(feat_imp_df.head(15).to_string(index=False))
    print("=" * 70)
    
    # 시각화
    fig = go.Figure()
    
    top_n = 15
    top_features = feat_imp_df.head(top_n)
    
    fig.add_trace(go.Bar(
        x=top_features['Importance'],
        y=top_features['Feature'],
        orientation='h',
        marker=dict(
            color=top_features['Importance'],
            colorscale='Blues',
            showscale=True,
            colorbar=dict(title='Importance')
        )
    ))
    
    fig.update_layout(
        title=f'Top {top_n} Feature Importance ({final_model_name})',
        xaxis_title='Importance',
        yaxis_title='Feature',
        height=600,
        yaxis={'categoryorder':'total ascending'}
    )
    
    fig.show()
    
    print("\\n📊 Feature Importance 분석 완료")

---

## 9. Feature Importance 분석

### Top 특성 시각화

In [ ]:
# Test Set에 Traffic Light 적용 (Validation에서 결정된 임계값 사용)
traffic_light_test = pd.Series(y_test_prob).apply(
    lambda x: assign_traffic_light(x, red_threshold, yellow_threshold)
)

# 등급별 성능 분석
traffic_summary_test = []
for grade in ['Red', 'Yellow', 'Green']:
    mask = (traffic_light_test == grade)
    count = mask.sum()
    bankruptcy_count = y_test.values[mask].sum()
    bankruptcy_rate = y_test.values[mask].mean() if count > 0 else 0
    
    traffic_summary_test.append({
        '등급': grade,
        '기업 수': count,
        '비율': count / len(y_test),
        '실제 부도': bankruptcy_count,
        '정밀도': bankruptcy_rate,
        '포착률': bankruptcy_count / y_test.sum() if y_test.sum() > 0 else 0
    })

traffic_df_test = pd.DataFrame(traffic_summary_test)

print("=" * 70)
print("🚦 Traffic Light 시스템 (Test Set 최종 평가)")
print("=" * 70)
print(f"Red 임계값:    {red_threshold:.4f} (Validation에서 결정)")
print(f"Yellow 임계값: {yellow_threshold:.4f} (Validation에서 결정)")
print("=" * 70)
print(traffic_df_test.to_string(index=False))
print("=" * 70)

# 리스크 방어율
risk_captured_test = traffic_df_test[traffic_df_test['등급'].isin(['Red', 'Yellow'])]['포착률'].sum()
print(f"\n✅ 리스크 방어율: {risk_captured_test:.1%} (Red+Yellow에서 부도 포착)")
print("=" * 70)

### 8.2 Traffic Light 시스템 Test Set 평가

In [ ]:
# Test Set 예측 (Validation에서 결정된 모델 사용)
y_test_prob = final_model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_prob >= selected_threshold).astype(int)

# 메트릭 계산
test_pr_auc = average_precision_score(y_test, y_test_prob)
test_roc_auc = roc_auc_score(y_test, y_test_prob)
test_f2_score = fbeta_score(y_test, y_test_pred, beta=2)

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
test_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
test_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
test_type2_error = fn / (tp + fn) if (tp + fn) > 0 else 0

print("=" * 70)
print("🎯 Test Set 최종 평가 결과")
print("=" * 70)
print(f"⚠️  이 결과는 최종 보고용이며, 이전 단계에서 Test Set을")
print(f"   절대 사용하지 않았음을 보장합니다.")
print(f"\n사용 모델:      {final_model_name}")
print(f"사용 임계값:    {selected_threshold:.4f} (Validation에서 결정)")
print("=" * 70)
print(f"\nPR-AUC:         {test_pr_auc:.4f}")
print(f"ROC-AUC:        {test_roc_auc:.4f}")
print(f"F2-Score:       {test_f2_score:.4f}")
print(f"Precision:      {test_precision:.2%}")
print(f"Recall:         {test_recall:.2%}")
print(f"Type II Error:  {test_type2_error:.2%}")
print(f"\nConfusion Matrix:")
print(f"  TN: {tn:,}  |  FP: {fp:,}")
print(f"  FN: {fn:,}  |  TP: {tp:,}")
print("=" * 70)

### 8.1 Test Set 예측 (Validation에서 결정된 모델 및 임계값 적용)

---

## 8. 🎯 Test Set 최종 평가 (단 한 번만!) 🎯

### ⚠️⚠️⚠️ 중요한 공지 ⚠️⚠️⚠️

```
이 섹션부터 Test Set을 처음으로 사용합니다!

✅ 지금까지의 모든 의사결정:
   - 리샘플링 전략 선택
   - 모델 선택
   - 하이퍼파라미터 튜닝
   - 임계값 최적화
   - Traffic Light 임계값
   → 모두 Train/Validation Set에서만 수행됨!

✅ Test Set 사용 목적:
   - 최종 일반화 성능 확인
   - 실전 배포 성능 추정
   - 의사결정 아님, 평가만!

⚠️  이 결과를 보고 모델을 다시 튜닝하면 Data Leakage!
```

In [ ]:
# Validation Set에 Traffic Light 적용 (⚠️ Test Set 사용 안 함!)
def assign_traffic_light(prob, red_th, yellow_th):
    if prob >= red_th:
        return 'Red'
    elif prob >= yellow_th:
        return 'Yellow'
    else:
        return 'Green'

# Traffic Light 등급 할당
traffic_light_val = pd.Series(y_val_prob_final).apply(
    lambda x: assign_traffic_light(x, red_threshold, yellow_threshold)
)

# 등급별 성능 분석
traffic_summary = []
for grade in ['Red', 'Yellow', 'Green']:
    mask = (traffic_light_val == grade)
    count = mask.sum()
    bankruptcy_count = y_val[mask].sum()
    bankruptcy_rate = y_val[mask].mean() if count > 0 else 0
    
    traffic_summary.append({
        '등급': grade,
        '기업 수': count,
        '비율': count / len(y_val),
        '실제 부도': bankruptcy_count,
        '정밀도': bankruptcy_rate,
        '포착률': bankruptcy_count / y_val.sum() if y_val.sum() > 0 else 0
    })

traffic_df = pd.DataFrame(traffic_summary)

print("=" * 70)
print("🚦 Traffic Light 시스템 성능 (Validation Set)")
print("=" * 70)
print(traffic_df.to_string(index=False))
print("=" * 70)

# 리스크 방어율
risk_captured = traffic_df[traffic_df['등급'].isin(['Red', 'Yellow'])]['포착률'].sum()
print(f"\n✅ 리스크 방어율: {risk_captured:.1%} (Red+Yellow에서 부도 포착)")
print("=" * 70)

### 7.2 Traffic Light 성능 검증 (Validation Set)

In [ ]:
# Traffic Light 임계값 계산 (Validation Set 기반)
# Red: Recall 80% 보장 (이미 계산됨)
red_threshold = recall_80_threshold if recall_80_threshold is not None else selected_threshold

# Yellow: Recall 95% 보장
target_recall_yellow = 0.95
idx_recall_95 = np.where(recalls_val[:-1] >= target_recall_yellow)[0]

if len(idx_recall_95) > 0:
    recall_95_idx = idx_recall_95[np.argmax(precisions_val[:-1][idx_recall_95])]
    yellow_threshold = thresholds_val[recall_95_idx]
    yellow_precision = precisions_val[recall_95_idx]
    yellow_recall = recalls_val[recall_95_idx]
else:
    # Recall 95% 달성 불가능하면 가장 낮은 임계값 사용
    yellow_threshold = thresholds_val[-1] if len(thresholds_val) > 0 else 0.01
    yellow_precision = precisions_val[-1] if len(precisions_val) > 1 else 0
    yellow_recall = recalls_val[-1] if len(recalls_val) > 1 else 1

print("=" * 70)
print("🚦 Traffic Light 임계값 (Validation Set 기반)")
print("=" * 70)
print(f"🔴 Red (High Risk):   확률 >= {red_threshold:.4f}")
print(f"   목표: 부도 기업의 80% 포착")
print(f"\n🟡 Yellow (Medium):    {yellow_threshold:.4f} <= 확률 < {red_threshold:.4f}")
print(f"   목표: 부도 기업의 95% 포착 (Red+Yellow)")
print(f"\n🟢 Green (Low Risk):  확률 < {yellow_threshold:.4f}")
print("=" * 70)

### 7.1 Traffic Light 임계값 계산 (Validation Set)

---

## 7. ⭐ Traffic Light 시스템 (데이터 기반 임계값) ⭐

### 임계값 설정 논리 (Validation Set 기반)

```
🔴 Red (High Risk):    Recall 80% 보장 임계값 이상
🟡 Yellow (Medium):    Recall 95% 보장 임계값 이상
🟢 Green (Low Risk):   그 외
```

**논리적 근거**: 부도 기업의 80%는 Red, 95%는 Red+Yellow에서 포착

In [ ]:
# Recall 80% 이상 달성하는 임계값 중 Precision 최대
target_recall = 0.80
idx_recall_80 = np.where(recalls_val[:-1] >= target_recall)[0]

if len(idx_recall_80) > 0:
    recall_80_idx = idx_recall_80[np.argmax(precisions_val[:-1][idx_recall_80])]
    recall_80_threshold = thresholds_val[recall_80_idx]
    recall_80_precision = precisions_val[recall_80_idx]
    recall_80_recall = recalls_val[recall_80_idx]
    recall_80_f2 = f2_scores[recall_80_idx]
    
    print("=" * 70)
    print(f"📊 Recall {target_recall:.0%} 보장 임계값 (Validation Set 기반)")
    print("=" * 70)
    print(f"임계값:   {recall_80_threshold:.4f}")
    print(f"Precision: {recall_80_precision:.2%}")
    print(f"Recall:    {recall_80_recall:.2%}")
    print(f"F2-Score:  {recall_80_f2:.4f}")
    print("=" * 70)
else:
    recall_80_threshold = None
    print("⚠️  Recall 80% 달성 불가능")

# 최종 사용 임계값 선택
selected_threshold = recall_80_threshold if recall_80_threshold is not None else f2_optimal_threshold
selected_threshold_type = f"Recall {target_recall:.0%} 보장" if recall_80_threshold is not None else "F2-Score 최적"

print(f"\n✅ 최종 선택 임계값: {selected_threshold:.4f} ({selected_threshold_type})")
print("=" * 70)

### 6.2 Recall 80% 보장 임계값 (Validation Set)

In [ ]:
# Validation Set에서 Precision-Recall 커브 계산 (⚠️ Test Set 사용 안 함!)
precisions_val, recalls_val, thresholds_val = precision_recall_curve(y_val, y_val_prob_final)

# F2-Score 계산 (Recall 우선, beta=2)
beta = 2
f2_scores = (1 + beta**2) * (precisions_val[:-1] * recalls_val[:-1]) / \
            (beta**2 * precisions_val[:-1] + recalls_val[:-1] + 1e-10)

# F2-Score 최대 임계값
f2_optimal_idx = np.argmax(f2_scores)
f2_optimal_threshold = thresholds_val[f2_optimal_idx]
f2_optimal_score = f2_scores[f2_optimal_idx]
f2_optimal_precision = precisions_val[f2_optimal_idx]
f2_optimal_recall = recalls_val[f2_optimal_idx]

print("=" * 70)
print("📊 F2-Score 최적 임계값 (Validation Set 기반)")
print("=" * 70)
print(f"임계값:   {f2_optimal_threshold:.4f}")
print(f"F2-Score: {f2_optimal_score:.4f}")
print(f"Precision: {f2_optimal_precision:.2%}")
print(f"Recall:    {f2_optimal_recall:.2%}")
print("=" * 70)

### 6.1 F2-Score 최적 임계값 (Validation Set)

---

## 6. ⭐ 임계값 최적화 (Validation Set 기반) ⭐

### 핵심 원칙

```
✅ Validation Set에서만 임계값 최적화
✅ F2-Score 최적화 (Recall 우선)
✅ Recall 80% 목표 임계값도 계산
⚠️ Test Set에는 결정된 임계값을 "적용"만 함!
```

In [ ]:
# 최종 모델 선택 (단일 모델 우선)
final_model = tuning_results[best_val_model_name]['best_estimator']
final_model_name = best_val_model_name

print("=" * 70)
print("✅ 최종 모델 선택 완료")
print("=" * 70)
print(f"모델: {final_model_name}")
print(f"선택 근거: Validation Set에서 PR-AUC 최고 성능 ({best_val_pr_auc:.4f})")
print(f"추가 고려사항: Part 4 SHAP 분석을 위해 단일 모델 선택")
print("=" * 70)

# 선택된 모델의 Validation 확률 저장
y_val_prob_final = final_model.predict_proba(X_val)[:, 1]

print(f"\n📊 최종 모델의 Validation Set 성능:")
print(val_results_df[val_results_df['Model'] == final_model_name].T.to_string(header=False))
print("=" * 70)

### 5.2 최종 모델 선택

In [ ]:
print("=" * 70)
print("📊 전체 모델 Validation Set 평가 (⚠️ Test Set 사용 안 함!)")
print("=" * 70)

# Validation 평가 결과 저장
validation_results = []

for model_name, result in tuning_results.items():
    model = result['best_estimator']
    
    # Validation Set 예측 (⚠️ Test Set 절대 사용 안 함!)
    y_val_prob = model.predict_proba(X_val)[:, 1]
    y_val_pred = (y_val_prob >= 0.5).astype(int)  # 기본 임계값 0.5
    
    # 메트릭 계산
    pr_auc = average_precision_score(y_val, y_val_prob)
    roc_auc = roc_auc_score(y_val, y_val_prob)
    f2_score = fbeta_score(y_val, y_val_pred, beta=2)
    
    # Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(y_val, y_val_pred).ravel()
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    type2_error = fn / (tp + fn) if (tp + fn) > 0 else 0
    
    validation_results.append({
        'Model': model_name,
        'PR_AUC': pr_auc,
        'ROC_AUC': roc_auc,
        'F2_Score': f2_score,
        'Precision': precision,
        'Recall': recall,
        'Type_II_Error': type2_error,
        'TP': tp,
        'FP': fp,
        'TN': tn,
        'FN': fn
    })

# DataFrame으로 변환
val_results_df = pd.DataFrame(validation_results).sort_values('PR_AUC', ascending=False).reset_index(drop=True)

print(val_results_df[['Model', 'PR_AUC', 'F2_Score', 'Recall', 'Type_II_Error']].to_string(index=False))
print("=" * 70)

# 최고 성능 모델
best_val_model_name = val_results_df.iloc[0]['Model']
best_val_pr_auc = val_results_df.iloc[0]['PR_AUC']

print(f"\n✅ Validation Set 최고 성능 모델: {best_val_model_name}")
print(f"   PR-AUC: {best_val_pr_auc:.4f}")
print(f"   F2-Score: {val_results_df.iloc[0]['F2_Score']:.4f}")
print(f"   Recall: {val_results_df.iloc[0]['Recall']:.2%}")
print(f"   Type II Error: {val_results_df.iloc[0]['Type_II_Error']:.2%}")
print("=" * 70)

### 5.1 전체 모델 Validation Set 평가

---

## 5. ⭐ 모델 선택 (Validation Set 기반) ⭐

### 핵심 원칙

```
✅ Validation Set에서만 평가
✅ Test Set 절대 사용 안 함!
✅ PR-AUC, F2-Score, Recall 종합 평가
✅ 단일 모델 우선 (Part 4 SHAP 분석 고려)
```

**비유**: 모의고사(Validation)로 최종 모델을 선택하고, 수능(Test)은 단 한 번만!

In [ ]:
# 결과 요약 DataFrame
summary_df = pd.DataFrame({
    'Model': list(tuning_results.keys()),
    'CV_PR_AUC': [result['best_cv_score'] for result in tuning_results.values()]
}).sort_values('CV_PR_AUC', ascending=False).reset_index(drop=True)

print("=" * 70)
print("📊 모델별 CV PR-AUC 순위 (Train Set 5-Fold CV)")
print("=" * 70)
print(summary_df.to_string(index=False))
print("=" * 70)

# 최고 성능 모델
best_model_name = summary_df.iloc[0]['Model']
best_model = tuning_results[best_model_name]['best_estimator']
best_cv_score = summary_df.iloc[0]['CV_PR_AUC']

print(f"\n✅ 최고 성능 모델: {best_model_name}")
print(f"   CV PR-AUC: {best_cv_score:.4f}")
print("=" * 70)

# 시각화
fig = go.Figure()

fig.add_trace(go.Bar(
    x=summary_df['Model'],
    y=summary_df['CV_PR_AUC'],
    text=[f"{score:.4f}" for score in summary_df['CV_PR_AUC']],
    textposition='outside',
    marker=dict(
        color=summary_df['CV_PR_AUC'],
        colorscale='Blues',
        showscale=True,
        colorbar=dict(title='PR-AUC')
    )
))

fig.update_layout(
    title='모델별 CV PR-AUC 성능 비교 (Train Set 5-Fold CV)',
    xaxis_title='모델',
    yaxis_title='PR-AUC',
    height=500
)

fig.show()

print("\n⚠️  이 결과는 Train Set CV 기반입니다. 최종 모델 선택은 Validation Set에서 진행합니다!")

### 4.3 튜닝 결과 요약

In [ ]:
# 모델 및 파라미터 그리드 정의
models_to_tune = {
    'LightGBM': (lgb.LGBMClassifier(random_state=RANDOM_STATE, verbose=-1), lgbm_param_grid),
    'XGBoost': (xgb.XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss'), xgb_param_grid),
    'CatBoost': (CatBoostClassifier(random_state=RANDOM_STATE, verbose=0), catboost_param_grid),
    'LogisticRegression': (LogisticRegression(random_state=RANDOM_STATE, max_iter=1000), lr_param_grid)
}

# 튜닝 결과 저장
tuning_results = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scorer = make_scorer(average_precision_score, needs_proba=True)

print("=" * 70)
print("AutoML 하이퍼파라미터 튜닝 시작 (Train + 5-Fold CV)")
print("=" * 70)

for model_name, (model, param_grid) in models_to_tune.items():
    print(f"\n🔄 {model_name} 튜닝 중...")
    start_time = datetime.now()
    
    # 파이프라인 생성
    pipeline = create_pipeline(model, use_resampler=selected_resampler)
    
    # 파라미터 그리드를 파이프라인 형식으로 변환
    pipeline_param_grid = {f'classifier__{k}': v for k, v in param_grid.items()}
    
    # RandomizedSearchCV
    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=pipeline_param_grid,
        n_iter=30,  # 빠른 탐색을 위해 30회로 제한
        scoring=scorer,
        cv=cv,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=0
    )
    
    # Train Set에서만 학습 (⚠️ Test Set 사용 안 함!)
    search.fit(X_train, y_train)
    
    # 결과 저장
    tuning_results[model_name] = {
        'best_estimator': search.best_estimator_,
        'best_params': search.best_params_,
        'best_cv_score': search.best_score_,  # CV PR-AUC
        'cv_results': search.cv_results_
    }
    
    elapsed = (datetime.now() - start_time).total_seconds()
    print(f"   ✅ 완료 ({elapsed:.1f}초)")
    print(f"   최고 CV PR-AUC: {search.best_score_:.4f}")
    print(f"   최적 파라미터: {search.best_params_}")

print("\n" + "=" * 70)
print("AutoML 튜닝 완료")
print("=" * 70)

### 4.2 모델별 튜닝 실행 (Train + CV만 사용)

**중요**: Test Set은 절대 사용하지 않습니다!

In [ ]:
# LightGBM 하이퍼파라미터 그리드 (간략화)
lgbm_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 63, 127],
    'min_child_samples': [20, 50, 100],
    'scale_pos_weight': [scale_pos_weight] if selected_resampler is None else [1]
}

# XGBoost 하이퍼파라미터 그리드
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'scale_pos_weight': [scale_pos_weight] if selected_resampler is None else [1]
}

# CatBoost 하이퍼파라미터 그리드
catboost_param_grid = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5],
    'scale_pos_weight': [scale_pos_weight] if selected_resampler is None else [1]
}

# Logistic Regression 하이퍼파라미터 그리드
lr_param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],
    'class_weight': ['balanced'] if selected_resampler is None else [None]
}

print("✅ 하이퍼파라미터 그리드 정의 완료")
print(f"   선택된 리샘플링 전략: {selected_strategy}")
print(f"   Class Weight 적용: {'Yes' if selected_resampler is None else 'No'}")

### 4.1 하이퍼파라미터 그리드 정의

---

## 4. AutoML: 하이퍼파라미터 튜닝

### 튜닝 전략

✅ **선택된 리샘플링 전략** 사용  
✅ **Train Set + 5-Fold CV**로만 학습  
✅ **RandomizedSearchCV** (빠른 탐색)  
✅ **PR-AUC** 최적화  
⚠️ **Test Set 절대 사용 안 함!**

### 모델 목록

- LightGBM (빠르고 효율적)
- XGBoost (강력한 성능)
- CatBoost (범주형 변수 처리 우수)
- Logistic Regression (설명력 우수, 베이스라인)

In [ ]:
# 전략 비교
print("=" * 70)
print("📊 리샘플링 전략 Validation 평가 결과")
print("=" * 70)

# Strategy A 최고 성능
best_smote = max(strategy_a_results.items(), key=lambda x: x['pr_auc'])
best_smote_name, best_smote_result = best_smote

print(f"\nStrategy A (SMOTE 계열):")
print(f"  최고 성능: {best_smote_name.upper()} - PR-AUC = {best_smote_result['pr_auc']:.4f}")

print(f"\nStrategy B (Class Weight):")
print(f"  PR-AUC = {pr_auc_b:.4f}")

# 성능 차이 계산
performance_diff = pr_auc_b - best_smote_result['pr_auc']
percentage_diff = (performance_diff / best_smote_result['pr_auc']) * 100

print(f"\n{'=' * 70}")
print(f"성능 차이: {performance_diff:+.4f} ({percentage_diff:+.2f}%)")
print(f"{'=' * 70}")

# 최종 선택
if pr_auc_b > best_smote_result['pr_auc']:
    selected_strategy = 'Class Weight'
    selected_resampler = None
    selected_model = pipeline_b
    selected_pr_auc = pr_auc_b
    reason = f"Class Weight가 {abs(performance_diff):.4f} 더 우수"
else:
    selected_strategy = f'SMOTE ({best_smote_name})'
    selected_resampler = best_smote_name
    selected_model = best_smote_result['model']
    selected_pr_auc = best_smote_result['pr_auc']
    reason = f"{best_smote_name}가 {abs(performance_diff):.4f} 더 우수"

print(f"\n✅ 최종 선택: {selected_strategy}")
print(f"   이유: {reason}")
print(f"   Validation PR-AUC: {selected_pr_auc:.4f}")
print("=" * 70)

### 3.4 전략 비교 및 최종 선택 (Validation 기반)

In [ ]:
print("=" * 70)
print("Strategy B: Class Weight Only")
print("=" * 70)

# LightGBM with Class Weight
lgbm_weighted = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.05,
    scale_pos_weight=scale_pos_weight,  # ⭐ 클래스 가중치 적용
    random_state=RANDOM_STATE,
    verbose=-1
)

# 파이프라인 생성 (리샘플링 없음)
pipeline_b = create_pipeline(lgbm_weighted, use_resampler=None)

# Train set 학습
pipeline_b.fit(X_train, y_train)

# Validation set 평가 (⚠️ Test set 사용 안 함!)
y_val_prob_b = pipeline_b.predict_proba(X_val)[:, 1]
pr_auc_b = average_precision_score(y_val, y_val_prob_b)

print(f"\n✅ Class Weight Only Validation PR-AUC: {pr_auc_b:.4f}")
print("=" * 70)

### 3.3 Strategy B: Class Weight Only (리샘플링 없음)

In [ ]:
print("=" * 70)
print("Strategy A: SMOTE 계열 테스트")
print("=" * 70)

# 기본 LightGBM 모델 (Class Weight 없음)
lgbm_base = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.05,
    random_state=RANDOM_STATE,
    verbose=-1
)

# SMOTE 변형 테스트
smote_variants = ['smote', 'borderline', 'smote_tomek']
strategy_a_results = {}

for variant in smote_variants:
    print(f"\n🔄 {variant.upper()} 테스트 중...")
    
    # 파이프라인 생성
    pipeline = create_pipeline(lgbm_base, use_resampler=variant)
    
    # Train set 학습
    pipeline.fit(X_train, y_train)
    
    # Validation set 평가 (⚠️ Test set 사용 안 함!)
    y_val_prob = pipeline.predict_proba(X_val)[:, 1]
    pr_auc = average_precision_score(y_val, y_val_prob)
    
    strategy_a_results[variant] = {
        'model': pipeline,
        'pr_auc': pr_auc
    }
    
    print(f"   ✅ Validation PR-AUC: {pr_auc:.4f}")

print("\n" + "=" * 70)
print("Strategy A 결과 요약:")
print("=" * 70)
for variant, result in strategy_a_results.items():
    print(f"{variant.upper():15s}: PR-AUC = {result['pr_auc']:.4f}")
print("=" * 70)

### 3.2 Strategy A: SMOTE 계열 (Class Weight 없음)

In [ ]:
# 불균형 비율 계산 (Class Weight용)
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight = neg_count / pos_count

print(f"📊 Train Set 불균형 정보:")
print(f"   정상 기업: {neg_count:,}개")
print(f"   부도 기업: {pos_count:,}개")
print(f"   불균형 비율: 1:{scale_pos_weight:.1f}")
print(f"   scale_pos_weight: {scale_pos_weight:.2f}")

### 3.1 불균형 비율 계산

---

## 3. 리샘플링 전략 대조 실험 ⭐

### 핵심 질문

**SMOTE vs Class Weight, 어느 것이 더 효과적인가?**

- **Strategy A**: SMOTE 계열 (Class Weight 없음)
- **Strategy B**: Class Weight Only (리샘플링 없음)

### 실험 설계

✅ **Train Set**으로 학습  
✅ **Validation Set**으로 평가 (Test set 절대 사용 안 함!)  
✅ **PR-AUC**로 비교 (불균형 데이터 핵심 지표)

In [ ]:
def create_pipeline(classifier, use_resampler=None):
    """
    전처리 + 리샘플링 + 분류기 파이프라인 생성
    
    Parameters:
    -----------
    classifier : estimator
        분류 모델
    use_resampler : str or None
        리샘플링 방법 ('smote', 'borderline', 'smote_tomek', None)
    
    Returns:
    --------
    pipeline : ImbPipeline
        전체 파이프라인
    """
    
    # 리샘플러 선택
    resampler_map = {
        'smote': SMOTE(sampling_strategy=0.2, random_state=RANDOM_STATE),
        'borderline': BorderlineSMOTE(sampling_strategy=0.2, random_state=RANDOM_STATE),
        'smote_tomek': SMOTETomek(sampling_strategy=0.2, random_state=RANDOM_STATE),
        None: 'passthrough'
    }
    
    resampler = resampler_map.get(use_resampler, 'passthrough')
    
    # 파이프라인 구성
    pipeline = ImbPipeline([
        ('inf_handler', InfiniteHandler()),
        ('imputer', SimpleImputer(strategy='median')),
        ('log_transformer', LogTransformer()),
        ('scaler', RobustScaler()),
        ('resampler', resampler),
        ('classifier', classifier)
    ])
    
    return pipeline

print("✅ 파이프라인 생성 함수 정의 완료")
print("   순서: InfiniteHandler → Imputer → LogTransformer → RobustScaler → Resampler → Classifier")

### 2.2 파이프라인 생성 함수

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class InfiniteHandler(BaseEstimator, TransformerMixin):
    """무한대 값을 처리하는 변환기"""
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        # 무한대 값을 NaN으로 변환
        X_copy = X_copy.replace([np.inf, -np.inf], np.nan)
        return X_copy

class LogTransformer(BaseEstimator, TransformerMixin):
    """양수 특성에 로그 변환 적용"""
    def __init__(self, epsilon=1e-10):
        self.epsilon = epsilon
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        # 모든 값이 0 이상인 특성에만 로그 변환
        for col in X_copy.columns:
            if (X_copy[col] >= 0).all():
                X_copy[col] = np.log1p(X_copy[col] + self.epsilon)
        return X_copy

print("✅ 커스텀 전처리 클래스 정의 완료")
print("   - InfiniteHandler: 무한대 값 처리")
print("   - LogTransformer: 로그 변환 (양수 특성만)")

# 📗 발표용 Part 3: 모델링 및 최적화 v2

---

## 🎯 Part 3 목표 및 이전 Part 요약

### Part 1-2 완료 사항

**Part 1: 문제 정의 및 핵심 발견**
- 50,105개 기업, 부도율 1.51% (1:66 불균형)
- **유동성**이 가장 강력한 예측 변수 발견
- 업종별 부도율 최대 2배 차이 (건설업 2.1% vs 제조업 1.1%)

**Part 2: 도메인 특성 공학**
- 52개 도메인 기반 특성 생성
- VIF/IV/AUC 기반 검증 → **27개 최종 선택**
- `domain_based_features_완전판.csv` 출력

### Part 3 핵심 목표

1. ✅ **Data Leakage 완전 제거** - Test set은 최종 평가 단 한 번만
2. ✅ **3-Way Split** (Train 60% / Validation 20% / Test 20%)
3. ✅ **리샘플링 전략 대조 실험** (SMOTE vs Class Weight)
4. ✅ **Validation 기반 모든 의사결정** (모델 선택, 임계값 최적화)
5. ✅ **F2-Score 최적화** (Recall 우선, Type II Error 최소화)
6. ✅ **비즈니스 임팩트 입증** (Traffic Light 시스템)

### ⚠️ Critical Requirements

```
❌ 절대 금지:
   - Test set으로 모델 선택
   - Test set으로 임계값 최적화
   - Test set으로 Traffic Light 기준 결정
   
✅ 필수:
   - Test set은 최종 보고 직전 단 한 번만 평가
   - 모든 의사결정은 Train/Validation에서만
   - Validation set 또는 CV로 임계값 최적화
```

---

## 0. 환경 설정

### 라이브러리 임포트

In [ ]:
# 데이터 처리
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# 한글 폰트 설정
import platform
if platform.system() == 'Darwin':  # macOS
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:  # Linux
    plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

# 머신러닝
from sklearn.model_selection import (
    train_test_split, 
    StratifiedKFold, 
    cross_val_score,
    cross_val_predict,
    RandomizedSearchCV
)
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
    roc_curve,
    fbeta_score,
    make_scorer
)

# 불균형 처리
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import Pipeline as ImbPipeline

# 모델
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

# 통계
from scipy.stats import wilcoxon

# 유틸리티
import joblib
import os
from datetime import datetime

# 재현성 설정
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ 라이브러리 임포트 완료")
print(f"⏰ 실행 시각: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

### 경로 설정

In [ ]:
# 데이터 경로 (절대 경로 하드코딩 금지!)
DATA_DIR = '../data'
FEATURES_DIR = os.path.join(DATA_DIR, 'features')
PROCESSED_DIR = os.path.join(DATA_DIR, 'processed')

# 출력 파일 경로
os.makedirs(PROCESSED_DIR, exist_ok=True)

# 파일명 설정
INPUT_FILE = os.path.join(FEATURES_DIR, 'domain_based_features_완전판.csv')
OUTPUT_PREFIX = 'Part3_v2'

print(f"✅ 입력 파일: {INPUT_FILE}")
print(f"✅ 출력 디렉토리: {PROCESSED_DIR}")

---

## 1. 데이터 로딩 및 3-Way Split ⭐

### 핵심 원칙

```
전체 데이터 (50,105)
├─ Train Set (60%, ~30,063): 모델 학습 + CV 튜닝
├─ Validation Set (20%, ~10,021): 모델 선택, 임계값 최적화, 의사결정
└─ Test Set (20%, ~10,021): 최종 평가만 (절대 건드리지 않음!)
```

**비유**: Test set은 "봉인된 시험지". 모델 개발이 완전히 끝난 후 한 번만 개봉.

### 1.1 데이터 로딩

In [ ]:
# 데이터 로딩
df = pd.read_csv(INPUT_FILE, encoding='utf-8')

print(f"📊 데이터 형태: {df.shape}")
print(f"\n📋 컬럼 목록 ({len(df.columns)}개):")
print(df.columns.tolist())

# 타겟 변수 확인
TARGET_COL = '모형개발용Performance(향후1년내부도여부)'
if TARGET_COL not in df.columns:
    raise ValueError(f"타겟 변수 '{TARGET_COL}'를 찾을 수 없습니다!")

print(f"\n🎯 타겟 변수: {TARGET_COL}")
print(f"   부도 기업: {df[TARGET_COL].sum():,}개 ({df[TARGET_COL].mean():.2%})")
print(f"   정상 기업: {(~df[TARGET_COL].astype(bool)).sum():,}개 ({(1-df[TARGET_COL].mean()):.2%})")
print(f"   불균형 비율: 1:{int(1/df[TARGET_COL].mean())}")

### 1.2 특성-타겟 분리

In [ ]:
# 특성-타겟 분리
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

print(f"✅ 특성 개수: {X.shape[1]}개")
print(f"✅ 샘플 수: {len(X):,}개")
print(f"\n📊 특성 목록:")
for i, col in enumerate(X.columns, 1):
    print(f"   {i:2d}. {col}")

### 1.3 ⭐ 3-Way Split (Train/Validation/Test)

**중요**: 이 단계에서 분할한 Test set은 **노트북 마지막 섹션 전까지 절대 사용하지 않습니다!**

In [ ]:
# 1차 분할: Train+Val (80%) vs Test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=RANDOM_STATE
)

# 2차 분할: Train (75% of 80% = 60%) vs Val (25% of 80% = 20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, 
    test_size=0.25, 
    stratify=y_temp, 
    random_state=RANDOM_STATE
)

print("="*70)
print("✅ 3-Way Split 완료")
print("="*70)
print(f"Train Set:      {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%, 부도율: {y_train.mean():.2%})")
print(f"Validation Set: {len(X_val):,} ({len(X_val)/len(X)*100:.1f}%, 부도율: {y_val.mean():.2%})")
print(f"Test Set:       {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%, 부도율: {y_test.mean():.2%})")
print("="*70)
print("⚠️  Test Set은 최종 평가 전까지 절대 사용하지 않음!")
print("="*70)

# 부도 기업 수 확인
print(f"\n📊 부도 기업 분포:")
print(f"   Train:      {y_train.sum():,}개")
print(f"   Validation: {y_val.sum():,}개")
print(f"   Test:       {y_test.sum():,}개")
print(f"   합계:       {y.sum():,}개 ✓")

### 1.4 데이터 분할 시각화

In [ ]:
# 데이터 분할 시각화
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['데이터 분할 비율', '부도율 분포'],
    specs=[[{'type': 'pie'}, {'type': 'bar'}]]
)

# 파이 차트: 데이터 분할
fig.add_trace(
    go.Pie(
        labels=['Train (60%)', 'Validation (20%)', 'Test (20%)'],
        values=[len(X_train), len(X_val), len(X_test)],
        marker=dict(colors=['#3498db', '#f39c12', '#e74c3c']),
        textposition='inside',
        textinfo='label+percent+value'
    ),
    row=1, col=1
)

# 바 차트: 부도율
bankruptcy_rates = [y_train.mean(), y_val.mean(), y_test.mean()]
fig.add_trace(
    go.Bar(
        x=['Train', 'Validation', 'Test'],
        y=bankruptcy_rates,
        marker=dict(color=['#3498db', '#f39c12', '#e74c3c']),
        text=[f"{rate:.2%}" for rate in bankruptcy_rates],
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='3-Way Split 결과 (Stratified)',
    height=400,
    showlegend=False
)

fig.update_yaxes(title_text='부도율', tickformat='.1%', row=1, col=2)

fig.show()

print("\n✅ 모든 세트에서 부도율이 균일하게 유지됨 (Stratified Split 성공)")

---

## 2. 전처리 파이프라인 정의

### 핵심 개선 사항

1. **순서 최적화**: Imputer → Log Transform → Scaler
2. **Winsorizer 제거**: Tree 모델은 이상치에 강건, 극단값은 부도 시그널일 수 있음
3. **RobustScaler 사용**: 이상치에 강건한 스케일링
4. **클래스 가중치 우선**: SMOTE보다 안정적

### 2.1 커스텀 전처리 클래스 정의